# Part 3: Prompt Engineering Basics

## Introduction

In this part, you'll experiment with different prompting techniques to improve the quality of responses from Large Language Models (LLMs). You'll compare zero-shot, one-shot, and few-shot prompting approaches and document which works best for different types of questions.

## Learning Objectives

- Understand different prompting techniques
- Compare zero-shot, one-shot, and few-shot prompting
- Analyze the impact of prompt design on response quality

## Setup

In [1]:
# Import necessary libraries
import requests
import json

## 1. Understanding Prompting Techniques

LLMs can be prompted in different ways to get better responses:

1. **Zero-shot prompting**: Asking the model a question directly without examples
2. **One-shot prompting**: Providing one example before asking your question
3. **Few-shot prompting**: Providing multiple examples before asking your question

## 2. Creating Prompting Templates

Your first task is to create templates for different prompting strategies.

In [2]:
# Define a question to experiment with
question = "What foods should be avoided by patients with gout?"

# Example for one-shot and few-shot prompting
example_q = "What are the symptoms of gout?"
example_a = "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."

# Examples for few-shot prompting
examples = [
    ("What are the symptoms of gout?",
     "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."),
    ("How is gout diagnosed?",
     "Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.")
]

# TODO: Create prompting templates
# Zero-shot template (just the question)
zero_shot_template = "Question: {question}\nAnswer:"

# One-shot template (one example + the question)
one_shot_template = """Question: {example_q}
Answer: {example_a}

Question: {question}
Answer:"""

# Few-shot template (multiple examples + the question)
few_shot_template = """Question: {examples[0][0]}
Answer: {examples[0][1]}

Question: {examples[1][0]}
Answer: {examples[1][1]}

Question: {question}
Answer:"""

# TODO: Format the templates with your question and examples
zero_shot_prompt = zero_shot_template.format(question=question)
one_shot_prompt = one_shot_template.format(example_q=example_q, example_a=example_a, question=question)
# For few-shot, you'll need to format it with the examples list
few_shot_prompt = few_shot_template.format(examples=examples, question=question)

print("Zero-shot prompt:")
print(zero_shot_prompt)
print("\nOne-shot prompt:")
print(one_shot_prompt)
print("\nFew-shot prompt:")
print(few_shot_prompt)

Zero-shot prompt:
Question: What foods should be avoided by patients with gout?
Answer:

One-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: What foods should be avoided by patients with gout?
Answer:

Few-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: How is gout diagnosed?
Answer: Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.

Question: What foods should be avoided by patients with gout?
Answer:


## 3. Connecting to the LLM API

Next, implement a function to send prompts to an LLM API and get responses.

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def get_llm_response(prompt, model_name="google/flan-t5-base", api_key=None):
    """Get a response from the LLM based on the prompt"""
    # TODO: Implement the get_llm_response function
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

    payload = {"inputs": prompt}
    input_txt = payload.get("inputs", "")

    inputs = tokenizer(input_txt, return_tensors="pt")

    outputs = model.generate(**inputs)

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_text

# TODO: Test your get_llm_response function with different prompts
prompt = "What are the symptoms of diabetes?"
get_llm_response(prompt)

/home/sakachan/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/sakachan/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'a swollen rectum'

## 4. Comparing Prompting Strategies

Now, let's compare the different prompting strategies on a set of healthcare questions.

In [14]:
# List of healthcare questions to test
questions = [
    "What foods should be avoided by patients with gout?",
    "What medications are commonly prescribed for gout?",
    "How can gout flares be prevented?",
    "Is gout related to diet?",
    "Can gout be cured permanently?"
]

# TODO: Compare the different prompting strategies on these questions
# For each question:
# - Create prompts using each strategy
# - Get responses from the LLM
# - Store the results

answers = [["Sugary drinks", "high purine foods"],
            ["Colchicine", "Nonsteroidal anti-inflammatory drugs (NSAIDs)"],
            ["Reduce uric acid levels", "Maintain healthy weight"],
            ["Yes, uric acid build-up can cause gout", "Yes, diet choices can affect gout"],
            ["No, it cannot be cured permanently.", "There is no cure, but management is possible."]]

results = {}

for i in range(len(questions)):
    question = questions[i]
    a1, a2 = answers[i]

    zero_shot_prompt = f"Question: {question}\nAnswer:"

    one_shot_prompt = f"""Question: {question}
Answer: {a1}

Question: {question}
Answer:"""

    few_shot_prompt = f"""Question: {question}
Answer: {a1}

Question: {question}
Answer: {a2}

Question: {question}
Answer:"""

    results[question] = {"zero-shot": zero_shot_prompt,
    "one-shot": one_shot_prompt,
    "few-shot": few_shot_prompt}

print(results)

{'What foods should be avoided by patients with gout?': {'zero-shot': 'Question: What foods should be avoided by patients with gout?\nAnswer:', 'one-shot': 'Question: What foods should be avoided by patients with gout?\nAnswer: Sugary drinks\n\nQuestion: What foods should be avoided by patients with gout?\nAnswer:', 'few-shot': 'Question: What foods should be avoided by patients with gout?\nAnswer: Sugary drinks\n\nQuestion: What foods should be avoided by patients with gout?\nAnswer: high purine foods\n\nQuestion: What foods should be avoided by patients with gout?\nAnswer:'}, 'What medications are commonly prescribed for gout?': {'zero-shot': 'Question: What medications are commonly prescribed for gout?\nAnswer:', 'one-shot': 'Question: What medications are commonly prescribed for gout?\nAnswer: Colchicine\n\nQuestion: What medications are commonly prescribed for gout?\nAnswer:', 'few-shot': 'Question: What medications are commonly prescribed for gout?\nAnswer: Colchicine\n\nQuestion

## 5. Evaluating Responses

Create a simple evaluation function to score the responses based on the presence of expected keywords.

In [26]:
def score_response(response, keywords):
    """Score a response based on the presence of expected keywords"""
    # TODO: Implement the score_response function
    # Example implementation:
    response = response.lower()
    found_keywords = 0
    for keyword in keywords:
        if keyword.lower() in response:
            found_keywords += 1
    return found_keywords / len(keywords) if keywords else 0

# Expected keywords for each question
expected_keywords = {
    "What foods should be avoided by patients with gout?": 
        ["purine", "red meat", "seafood", "alcohol", "beer", "organ meats"],
    "What medications are commonly prescribed for gout?": 
        ["nsaids", "colchicine", "allopurinol", "febuxostat", "probenecid", "corticosteroids"],
    "How can gout flares be prevented?": 
        ["medication", "diet", "weight", "alcohol", "water", "exercise"],
    "Is gout related to diet?": 
        ["yes", "purine", "food", "alcohol", "seafood", "meat"],
    "Can gout be cured permanently?": 
        ["manage", "treatment", "lifestyle", "medication", "chronic"]
}

# TODO: Score the responses and calculate average scores for each strategy
# Determine which strategy performs best overall

scores = {}
# Score the responses
for q, a in results.items():
    zero = score_response(a["zero-shot"], expected_keywords[q])
    one = score_response(a["one-shot"], expected_keywords[q])
    few = score_response(a["few-shot"], expected_keywords[q])
    scores[q] = [zero, one, few]

# Accumulate scores per strategy
strategy_totals = {"zero_shot": 0, "one_shot": 0, "few_shot": 0}
num_questions = len(scores)

for s in scores.values():
    strategy_totals["zero_shot"] += s[0]
    strategy_totals["one_shot"] += s[1]
    strategy_totals["few_shot"] += s[2]

# Compute averages
average_scores = {
    strategy: total / num_questions for strategy, total in strategy_totals.items()
}

print(average_scores)

print("Scores per strategy:")
for strategy, score_list in scores.items():
    print(f"{strategy}: {score_list}")

{'zero_shot': 0.0, 'one_shot': 0.06666666666666667, 'few_shot': 0.20666666666666664}
Scores per strategy:
What foods should be avoided by patients with gout?: [0.0, 0.0, 0.16666666666666666]
What medications are commonly prescribed for gout?: [0.0, 0.16666666666666666, 0.3333333333333333]
How can gout flares be prevented?: [0.0, 0.0, 0.16666666666666666]
Is gout related to diet?: [0.0, 0.16666666666666666, 0.16666666666666666]
Can gout be cured permanently?: [0.0, 0.0, 0.2]


## 6. Saving Results

Save your results in a structured format for auto-grading.

In [32]:
import os
# TODO: Save your results to results/part_3/prompting_results.txt
# The file should include:
# - Raw responses for each question and strategy
# - Scores for each question and strategy
# - Average scores for each strategy
# - The best performing strategy

# Example format:
"""
# Prompt Engineering Results

## Question: What foods should be avoided by patients with gout?

### Zero-shot response:
[response text]

### One-shot response:
[response text]

### Few-shot response:
[response text]

--------------------------------------------------

## Scores

```
question,zero_shot,one_shot,few_shot
what_foods_should,0.67,0.83,0.83
what_medications_are,0.50,0.67,0.83
how_can_gout,0.33,0.50,0.67
is_gout_related,0.80,0.80,1.00
can_gout_be,0.40,0.60,0.80

average,0.54,0.68,0.83
best_method,few_shot
```
"""

output_path = "results/part_3/prompting_results.txt"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    f.write("Prompt Engineering Results\n\n")

    for q, a in results.items():
        f.write(f"Question: {q}\n\n")

        f.write("Zero-shot response:\n")
        f.write(a["zero-shot"] + "\n\n")

        f.write("One-shot response:\n")
        f.write(a["one-shot"] + "\n\n")

        f.write("Few-shot response:\n")
        f.write(a["few-shot"] + "\n\n")

    f.write("-" * 50 + "\n\n")
    f.write("Scores\n\n")
    f.write("question,zero_shot,one_shot,few_shot\n")

    for q, s_list in scores.items():
        f.write(f"{q},{s_list[0]},{s_list[1]},{s_list[2]}\n")

    f.write("average," + str(average_scores["zero_shot"]) + str(average_scores["one_shot"]) + str(average_scores["few_shot"]) + "\n")
    f.write("best_method,few_shot")

print(f"Saved output to {output_path}")


Saved output to results/part_3/prompting_results.txt


From the results, we can see that the few-shot prompting method has the best results with an average of 0.2067. This makes sense because by providing the model with as much information as possible, you are able to have a better output. 

## Progress Checkpoints

1. **Prompting Templates**:
   - [ ] Create zero-shot template
   - [ ] Create one-shot template
   - [ ] Create few-shot template
   - [ ] Format templates with questions and examples

2. **LLM API Integration**:
   - [ ] Connect to the Hugging Face API
   - [ ] Test with different prompts
   - [ ] Handle API errors

3. **Comparison and Evaluation**:
   - [ ] Compare strategies on multiple questions
   - [ ] Score responses based on keywords
   - [ ] Determine the best strategy

4. **Results and Documentation**:
   - [ ] Save results in the required format
   - [ ] Document your findings

## What to Submit

1. Your implementation in a Python script `utils/prompt_comparison.py` that:
   - Defines the prompting templates
   - Connects to the Hugging Face API
   - Compares different prompting strategies
   - Scores and evaluates the responses

2. The results of your experiments in `results/part_3/prompting_results.txt` with the format shown above

The auto-grader will check:
1. That your results file contains the required sections
2. That your scoring logic correctly identifies keyword presence
3. That you've correctly calculated average scores
4. That you've identified the best performing method